In [20]:
# ! ssh -fNL 27017:localhost:27017 bonnen@rxdhawkins.me

In [107]:
import datetime, os, json, boto, pymongo, numpy, h5py
import numpy as np
from sklearn.metrics import confusion_matrix
from boto.mturk.connection import MTurkConnection
a = np.array

In [108]:
live = 1
mturk_platform = 'live'
credential_locations = '../credentials/'

In [109]:
# bonus subject
host = 'mechanicalturk.amazonaws.com'
# extract account info and define platform
key_info = np.load(credential_locations + 'snail_rootkey.npy').item()
if live: host = 'mechanicalturk.amazonaws.com'
else: host = 'mechanicalturk.sandbox.amazonaws.com'
# establish mturk connection
mturk = MTurkConnection(aws_access_key_id=key_info['AWSAccessKeyId'],
                        aws_secret_access_key=key_info['AWSSecretKey'],
                        host=host)

In [110]:
# extract hit info from submission inventory
submission_file = 'HIT_submission_records_%s.npy'%(mturk_platform)
submission_records = np.load(submission_file).item()
submissions = submission_records.keys()

In [111]:
access_info = json.load(open(credential_locations + 'auth.json'))

# open tunnel -- requires established connection [ssh -fNL 27017:localhost:27017 bonnen@rxdhawkins.m]
mongo_tunnel = 'mongodb://' + access_info['user'] + ':' + access_info['password'] + '@127.0.0.1'

# conect, define collection and database
connection = pymongo.MongoClient(mongo_tunnel)
data_base = connection['task_stream']
collection = data_base['match_to_sample']

In [177]:

def get_hit_and_worker_ids(i_submission): 

    submission_info = submission_records[i_submission]
    hit_id = submission_info['hit_id']
    subject_ids = a(collection.find({'hit_id':hit_id}).distinct('worker_id'))
    
    return hit_id, subject_ids

def generate_record_for_inventory(subject_ids, submission_records, i_submission): 
    
    if len(subject_ids): 
        
        try: 
            submission_records[i_submission]['subject_info'] 
        except: 
            submission_records[i_submission]['subject_info'] = {} 
            #print('creating new submission inventory... ')

        for i_subject in subject_ids: 

            try: 
                submission_records[i_submission]['subject_info'][i_subject] 
            except: 
                submission_records[i_submission]['subject_info'][i_subject] = {} 
                #print('creating new subject inventory')
    else: 
        
        print('\n no subject data \n')
        
    return submission_records 

def bonus_protocol(subject_ids, hit_id, i_submission,manual=False, print_repeat=False, show_error=False): 
    
    w_data = {i['worker_id']: i for i in collection.find({'hit_id':hit_id, "data_type":"worker_feedback"})}  

    for i_subject in list(w_data.keys()): 

        assignment_id = w_data[i_subject]['assignment_id']
        bonus_amount =  float('%.2f'%w_data[i_subject]['total_bonus_earned'])
        converted_bonus = boto.mturk.price.Price(bonus_amount) 

        try: 
            submission_records[i_submission]['subject_info'][i_subject]
            if submission_records[i_submission]['subject_info'][i_subject]['assigned_bonus'] == 1 : 
                needs_bonus = 0 
            else: 
                needs_bonus = 1 
        except: 
            needs_bonus = 1  

        converted_bonus = performance_bonus = boto.mturk.price.Price(bonus_amount)

        if needs_bonus: 

            if manual:
                
                try: 
                    print('n trials:', w_data[i_subject]['n_trials'])
                    print('duration:', w_data[i_subject]['experiment_duration'])
                    print('blocks_completed:', w_data[i_subject]['n_block'])
                    print(w_data[i_subject]['worker_input'])
                    print('bonus amount: %f'%bonus_amount)
                except: 
                    print("no feedback data for subject")
                
                MSG = input("Enter a custom message? Pressing enter reverts to default. ")
                if len(MSG) == 0: MSG = "Thanks for participating in our study, and apologies for the delay!"
                BONUS = input("Amount to bonus worker? Pressing enter reverts to {} ".format(bonus_amount))
                if len(BONUS) == 0: BONUS = bonus_amount
                converted_bonus = boto.mturk.price.Price(BONUS) 
                
            else: 
                
                # set criteria for increasing subjects bonus regardless of their performance
                condition_a = (w_data[i_subject]['experiment_duration'] > 40) # > 40 minutes
                condition_b = (w_data[i_subject]['total_bonus_earned'] < 5)   # < 5 dollars
                condition_c = (w_data[i_subject]['n_trials'] > 300)           # > 300 trials 
                
                if condition_a * condition_b * condition_c: 
                    orig_bonus = float('%.2f'%w_data[i_subject]['total_bonus_earned'])
                    MSG = "Thanks for participating in our study! We added %.2f to your bonus"%(5 - orig_bonus)
                    BONUS = 5. 
                    print("upgraded bonus from %.2f to %.2f"%(orig_bonus, BONUS))
                else: 
                    MSG = "Thanks for participating in our study!"
                    BONUS = float('%.2f'%w_data[i_subject]['total_bonus_earned'])
                
                converted_bonus = boto.mturk.price.Price(BONUS) 

            try: 
                print('worker_id:', i_subject, 
                      ', experiment_duration:', np.round(w_data[i_subject]['experiment_duration'],2), 
                      ', n_trials:', w_data[i_subject]['n_trials'] ,  
                      ', bonus:', converted_bonus, end=' ')
                
                s = mturk.grant_bonus(i_subject, assignment_id,  converted_bonus, MSG);
                print('success!')
                submission_records[i_submission]['subject_info'][i_subject]['assigned_bonus'] = 1
                submission_records[i_submission]['subject_info'][i_subject]['bonus_amount'] = converted_bonus
                submission_records[i_submission]['subject_info'][i_subject]['return_message']  = {type(s): s}

            except Exception as e:

                ERROR = {type(e):e}
                submission_records[i_submission]['subject_info'][i_subject]['assigned_bonus'] = 0
                submission_records[i_submission]['subject_info'][i_subject]['bonus_amount'] = 0 
                submission_records[i_submission]['subject_info'][i_subject]['return_message']  = {type(e):e}
                
                print('ERROR') # me"}

            np.save(submission_file, submission_records)

        else: 

            if print_repeat: print('already bonused %s'%i_subject)
            
    return submission_records

In [178]:

sub_completed = 342
to_do = [i for i in submission_records.keys() if int(i[np.nonzero([j == '_' for j in i])[0][0]+1:]) > sub_completed]


In [134]:

for i_submission in to_do: 
    print('\n', i_submission, '\n')
    hit_id, subject_ids = get_hit_and_worker_ids(i_submission)
    submission_records = generate_record_for_inventory(subject_ids, submission_records, i_submission) 
    submission_records = bonus_protocol(subject_ids, hit_id, i_submission, manual=False, print_repeat=True) 



 submission_343 

already bonused A22N5ONOWPNZGB
already bonused A1PFE8T2LNWFLV


In [163]:
submission_keys = list(submission_records.keys())

amounts = []

for i_sub in submission_keys: 
    
    if 'subject_info' in submission_records[i_sub].keys(): 
        
        hit = submission_records[i_sub]['subject_info']

        s_bonuses = [hit[i_sub]['bonus_amount'] for i_sub in hit.keys() if 'bonus_amount' in hit[i_sub].keys()]
        amounts.extend(s_bonuses)

